In [1]:
!pip install requests

In [2]:
import json
import requests
import urllib.parse
base_url = 'https://itunes.apple.com/search'

artist = 'Ed Sheeran'

params = {
    'term': artist,
    'limit': 200,
    'media': 'music',
    'entity': 'song',
    'attribute': 'artistTerm'
}

response = requests.get(f"{base_url}?{urllib.parse.urlencode(params)}")

data = json.loads(response.text)
data = data['results']

# Все доступные ключи объекта
print(list(data[0].keys()))

['wrapperType', 'kind', 'artistId', 'collectionId', 'trackId', 'artistName', 'collectionName', 'trackName', 'collectionCensoredName', 'trackCensoredName', 'artistViewUrl', 'collectionViewUrl', 'trackViewUrl', 'previewUrl', 'artworkUrl30', 'artworkUrl60', 'artworkUrl100', 'collectionPrice', 'trackPrice', 'releaseDate', 'collectionExplicitness', 'trackExplicitness', 'discCount', 'discNumber', 'trackCount', 'trackNumber', 'trackTimeMillis', 'country', 'currency', 'primaryGenreName', 'isStreamable']


In [3]:
import dateutil.parser

# Преобразование строки с датой в объект даты
# **x - скопировать все поля из объекта x
data = map(lambda x: {**x, 'releaseDate': dateutil.parser.isoparse(x['releaseDate'])}, data)
data = list(data)
# Убираем лишние поля
data = map(lambda x: {
    'artistId': x['artistId'],
    'collectionId': x['collectionId'],
    'trackId': x['trackId'],
    'artistName': x['artistName'],
    'collectionName': x['collectionCensoredName'],
    'trackName': x['trackCensoredName'],
    'collectionPrice': x['collectionPrice'],
    'trackPrice': x['trackPrice'],
    'releaseDate': x['releaseDate'],
    'collectionExplicitness': x['collectionExplicitness'],
    'trackExplicitness': x['trackExplicitness'],
    'discCount': x['discCount'],
    'discNumber': x['discNumber'],
    'trackCount': x['trackCount'],
    'trackNumber': x['trackNumber'],
    'trackTimeMillis': x['trackTimeMillis'],
    'country': x['country'],
    'currency': x['currency'],
    'primaryGenreName': x['primaryGenreName'],
    'isStreamable': x['isStreamable']
}, data)
data = list(data)
print(list(data[0].keys()))

print(set(map(lambda x: x['artistName'], data)))

['artistId', 'collectionId', 'trackId', 'artistName', 'collectionName', 'trackName', 'collectionPrice', 'trackPrice', 'releaseDate', 'collectionExplicitness', 'trackExplicitness', 'discCount', 'discNumber', 'trackCount', 'trackNumber', 'trackTimeMillis', 'country', 'currency', 'primaryGenreName', 'isStreamable']
{'Ed Sheeran & Justin Bieber', 'Ed Sheeran, Chris Stapleton & Bruno Mars', 'Fireboy DML & Ed Sheeran', 'Ed Sheeran & Andrea Bocelli', 'N.E.R.D & Ed Sheeran', 'Ed Sheeran & Yelawolf', 'Ed Sheeran & Travis Scott', 'Ed Sheeran & Gary Lightbody', 'Ed Sheeran & Elton John', 'Ed Sheeran'}


In [4]:
print('Название исполнителя: ',artist)
print('Самые популярные жанры: ',set(map(lambda x: x['primaryGenreName'], data)))

Название исполнителя:  Ed Sheeran
Самые популярные жанры:  {'Rock', 'Soundtrack', 'Classical Crossover', 'Hip-Hop/Rap', 'Holiday', 'Dance', 'Pop', 'Singer/Songwriter'}


In [5]:
from itertools import groupby
from re import I

collection_id_extractor = lambda x: x['collectionId']

sorted_by_album = sorted(data, key=collection_id_extractor)
album_groups = {}
album_names = {}

for album_id, songs in groupby(sorted_by_album, key=collection_id_extractor):
    songs = list(songs)
    album_groups[album_id] = songs 
    album_names[album_id] = songs[0]['collectionName']

del sorted_by_album

I = 0
print("Количество альбомов: ")
for album_names in album_names.values():
    I = I + 1
print(I)

Количество альбомов: 
42


In [6]:
from itertools import groupby
from re import A
track_count_extractor = lambda x: x['trackCount']

sorted_by_track = sorted(data, key=track_count_extractor)
track_groups = {}
track_names = {}

for track_id, songs in groupby(sorted_by_track, key=track_count_extractor):
    songs = list(songs)
    track_groups[track_id] = songs 
    track_names[track_id] = songs[0]['trackName']


A = 0
print("Количество песен: ")
for track_names in track_names.values():
    A = A + 1
print(A)

Количество песен: 
17


In [7]:
def convert_millis(millis: int) -> str:
    total_seconds = millis // 1000
    seconds = total_seconds % 60
    minutes = (total_seconds // 60) % 60
    hours = (total_seconds // (60 * 60))
    return f"{hours}H {minutes}M {seconds}S"

time_extractor = lambda x: x['trackTimeMillis']

longest_song = max(data, key=time_extractor)
print('Самая длинная песня:', longest_song['trackName'], ' и её длительность: ', convert_millis(longest_song['trackTimeMillis']))

shortest_song = min(data, key=time_extractor)
print('Самая короткая песня:', shortest_song['trackName'], ' и её длительность: ', convert_millis(shortest_song['trackTimeMillis']))

Самая длинная песня: You Need Me, I Don't Need You (Live)  и её длительность:  0H 19M 46S
Самая короткая песня: Outro  и её длительность:  0H 2M 16S


In [8]:
import statistics
print( 'Общая длительность песен:', convert_millis( sum( list( map( lambda x: x['trackTimeMillis'], data)))))
print( 'Средняя длительность песен:', convert_millis( round( statistics.mean( list( map( lambda x : x['trackTimeMillis'], data))))))
print( 'Медиана длительности песен:', convert_millis( round( statistics.median( list( map( lambda x : x['trackTimeMillis'], data )))))) 

Общая длительность песен: 13H 30M 13S
Средняя длительность песен: 0H 4M 3S
Медиана длительности песен: 0H 3M 49S


In [9]:
price_extractor = lambda x: x['trackPrice']

firstprized_song = max(data, key=price_extractor)
print('Самая дорогая песня:', firstprized_song['trackName'], ' и её стоимость: ', firstprized_song['trackPrice'])

lastprized_song = min(data, key=price_extractor)
print('Самая дешёвая песня:', lastprized_song['trackName'], ' и её стоимость: ', lastprized_song['trackPrice'])

Самая дорогая песня: Thinking Out Loud  и её стоимость:  1.29
Самая дешёвая песня: I See Fire  и её стоимость:  -1.0


In [10]:
print( 'Общая стоимость песен: ', round(sum( list( map( lambda x: x['trackPrice'], data))),2))
print( 'Средняя стоимость песен: ', round(statistics.mean( list( map( lambda x: x['trackPrice'], data))), 2))
print( 'Медиана стоимости песен: ', round(statistics.median( list( map( lambda x: x['trackPrice'], data))), 2))

Общая стоимость песен:  238.48
Средняя стоимость песен:  1.19
Медиана стоимости песен:  1.29


In [11]:
for i in range(I):
    album_name = list( set( map( lambda x: x['collectionName'], data)))[i]
    album_data = list( filter( lambda x:album_name in x['collectionName'], data))

    
    print( 'Название альбома:', set(list( map( lambda x: x['collectionName'], album_data))))
    print( 'Количество песен:', len( list( map( lambda x: x['trackName'], album_data))))
    print( 'Самый популярный жанр:',  set(list( map( lambda x: x['primaryGenreName'], album_data))))
    print( 'Дата релиза альбома:',  list( map( lambda x: x['releaseDate'], album_data))[0])
    
    longest_song = max(data, key=time_extractor)
    print('Самая длинная песня:', longest_song['trackName'], ' и её длительность: ', convert_millis(longest_song['trackTimeMillis']))
    shortest_song = min(data, key=time_extractor)
    print('Самая короткая песня:', shortest_song['trackName'], ' и её длительность: ', convert_millis(shortest_song['trackTimeMillis']))
    
    print( 'Общая длительность песен:', convert_millis( sum( list( map( lambda x: x['trackTimeMillis'], album_data)))))
    print( 'Средняя длительность песен:', convert_millis( round( statistics.mean( list( map( lambda x : x['trackTimeMillis'], album_data))))))
    print( 'Медиана длительности песен:', convert_millis( round( statistics.median( list( map( lambda x : x['trackTimeMillis'], album_data))))))
    
    
    firstprized_song = max(data, key=price_extractor)
    print('Самая дорогая песня:', firstprized_song['trackName'], ' и её стоимость: ', firstprized_song['trackPrice'])
    lastprized_song = min(data, key=price_extractor)
    print('Самая дешёвая песня:', lastprized_song['trackName'], ' и её стоимость: ', lastprized_song['trackPrice'])

    print( 'оОбщая стоимость песен: ', round( sum( list( map( lambda x: x['trackPrice'], album_data ) ) ), 2 ))
    print( 'Средняя стоимость песен: ', round( statistics.mean( list( map( lambda x: x['trackPrice'], album_data ) ) ), 2 ))
    print( 'Медиана стоимости песен: ', round( statistics.median( list( map( lambda x: x['trackPrice'], album_data ) ) ), 2 ))

Название альбома: {"I Don't Care (Chronixx & Koffee Remix) - Single"}
Количество песен: 1
Самый популярный жанр: {'Pop'}
Дата релиза альбома: 2019-06-25 12:00:00+00:00
Самая длинная песня: You Need Me, I Don't Need You (Live)  и её длительность:  0H 19M 46S
Самая короткая песня: Outro  и её длительность:  0H 2M 16S
Общая длительность песен: 0H 3M 38S
Средняя длительность песен: 0H 3M 38S
Медиана длительности песен: 0H 3M 38S
Самая дорогая песня: Thinking Out Loud  и её стоимость:  1.29
Самая дешёвая песня: I See Fire  и её стоимость:  -1.0
оОбщая стоимость песен:  1.29
Средняя стоимость песен:  1.29
Медиана стоимости песен:  1.29
Название альбома: {'Me Before You (Original Motion Picture Soundtrack)'}
Количество песен: 1
Самый популярный жанр: {'Pop'}
Дата релиза альбома: 2014-06-20 12:00:00+00:00
Самая длинная песня: You Need Me, I Don't Need You (Live)  и её длительность:  0H 19M 46S
Самая короткая песня: Outro  и её длительность:  0H 2M 16S
Общая длительность песен: 0H 4M 16S
Средня